In [1]:
from pyqpanda import *
from pyqpanda.Visualization.circuit_draw import *

import math
import numpy as np
from fractions import Fraction

In [2]:
N = 21

## Tool Functions

In [187]:
def draw(prog, filename=''):
#     print(prog)
    
    dir_path = './circuits/8/'
    if filename != '':
        draw_qprog(prog, 'pic', filename=f'{dir_path}{filename}')

In [291]:
## x h cx tof barrier cu1 ry

BLACK = 'black'
WHITE = 'white'

class QuantumCircuit:
    def __init__(self, qubits, circuit=None):
        self.qubits = qubits
        self.circuit = circuit if circuit != None else create_empty_circuit()
        
        self.expand = 'no'
    
    # Single qubit gates
    
    def x(self, index):
        self.circuit << X(self.qubits[index])
        return self
    
    def h(self, index):
        self.circuit << H(self.qubits[index])
        return self
    
    # Multi qubits gates
    
    def cnot(self, control, target):
        if not isinstance(control, list):
            self.circuit << CNOT(self.qubits[control], self.qubits[target])
        else:
            if control[0] not in [BLACK, WHITE]:
                raise ValueError('neither BLACK or WHITE')
            
            if control[0] == WHITE:
                self.x(control[1])
                self.circuit << CNOT(self.qubits[control[1]], self.qubits[target])
                self.x(control[1])
            else:
                self.circuit << CNOT(self.qubits[control[1]], self.qubits[target])
        
        return self
    
    def tof(self, control1, control2, target):
        if not isinstance(control1, list):
            control1 = [BLACK, control1]
        if not isinstance(control2, list):
            control2 = [BLACK, control2]
            
        if (control1[0] not in [BLACK, WHITE]) or (control2[0] not in [BLACK, WHITE]):
            raise ValueError('neither BLACK or WHITE')
            
        if control1[0] == WHITE:
            self.x(control1[1])
        if control2[0] == WHITE:
            self.x(control2[1])
            
        if self.expand == 'no':
            self.circuit << Toffoli(self.qubits[control1[1]], self.qubits[control2[1]], self.qubits[target])
        elif self.expand == 'expand':
            self.circuit << H(self.qubits[target]) << CNOT(self.qubits[control2[1]], self.qubits[target]) \
                << T(self.qubits[target]).dagger() << CNOT(self.qubits[control1[1]], self.qubits[target]) \
                << T(self.qubits[target]) << CNOT(self.qubits[control2[1]], self.qubits[target]) \
                << T(self.qubits[target]).dagger() << CNOT(self.qubits[control1[1]], self.qubits[target]) \
                << T(self.qubits[target]) << H(self.qubits[target]) << T(self.qubits[control2[1]]) \
                << CNOT(self.qubits[control1[1]], self.qubits[control2[1]]) \
                << T(self.qubits[control2[1]]).dagger() << T(self.qubits[control1[1]]) \
                << CNOT(self.qubits[control1[1]], self.qubits[control2[1]])
        elif self.expand == 'approximate':
            self.circuit << RY(self.qubits[target], np.pi / 4) << CNOT(self.qubits[control2[1]], self.qubits[target]) \
                 << RY(self.qubits[target], np.pi / 4) << CNOT(self.qubits[control1[1]], self.qubits[target]) \
                 << RY(self.qubits[target], -np.pi / 4) << CNOT(self.qubits[control2[1]], self.qubits[target]) \
                 << RY(self.qubits[target], -np.pi / 4)
            
        if control1[0] == WHITE:
            self.x(control1[1])
        if control2[0] == WHITE:
            self.x(control2[1])
        
        return self
    
    # another
    
    def barrier(self, targets=None):
        targets = targets if targets != None else self.qubits
        self.circuit << BARRIER(targets)
        return self

In [293]:
tof = QCircuit(qubits)
tof.expand = 'expand'

tof.tof(0,1,2)
draw(tof.circuit, 'toffoli')

In [294]:
tof = QCircuit(qubits)
tof.expand = 'approximate'

tof.tof(0,1,2)
draw(tof.circuit, 'toffoli_approximate')

## Init Quantum Machine

In [5]:
class InitQMachine:
    def __init__(self, qubitsCount, cbitsCount, machineType = QMachineType.CPU):
        self.machine = init_quantum_machine(machineType)
        
        self.qubits = self.machine.qAlloc_many(qubitsCount)
        self.cbits = self.machine.cAlloc_many(cbitsCount)
        
        print(f'Init Quantum Machine with qubits:[{qubitsCount}] / cbits:[{cbitsCount}] Successfully')
    
    def __del__(self):
        destroy_quantum_machine(self.machine)

## Init Shor Circuit

### - init qMachine and qubits/cbits

In [6]:
phase_bits_qubits = 3
function_value_qubits = 5

total_qubits = phase_bits_qubits + function_value_qubits

In [7]:
init_machine = InitQMachine(total_qubits, phase_bits_qubits)

qubits = init_machine.qubits
cbits = init_machine.cbits

Init Quantum Machine with qubits:[8] / cbits:[3] Successfully


### - First Step: apply hadamard gates

In [9]:
hadamard_circuit = create_empty_circuit()

for i in range(phase_bits_qubits):
    hadamard_circuit << H(qubits[i])
    
hadamard_circuit << BARRIER(qubits)

draw(hadamard_circuit, 'hadamard')

### - Second Step: apply unitary Uf

In [254]:
# https://arxiv.org/pdf/1202.6614.pdf

### - Example for a = 2

In [215]:
unitary_circuit2 = QCircuit(qubits)

unitary_circuit2 = unitary_circuit2 \
    .cnot(2, 1) \
    .tof(0, 1, 6).tof([WHITE, 0], [WHITE, 1], 3) \
    .cnot(2, 1) \
    .tof([WHITE, 1], [BLACK, 2], 5).tof([WHITE, 0], [BLACK, 5], 7) \
    .tof(0, 5, 3).tof([WHITE, 1], [BLACK, 2], 5) \
    .tof([BLACK, 1], [WHITE, 2], 4).tof([WHITE, 0], [BLACK, 4], 5) \
    .cnot(0, 4).cnot(5, 4) \
    .barrier().circuit
    
draw(unitary_circuit2 , 'unitary2')

### - Example for a = 4

In [227]:
unitary_circuit4  = QCircuit(qubits)

unitary_circuit4 = unitary_circuit4 \
    .cnot(2, 1) \
    .tof([BLACK, 0], [WHITE, 1], 5).tof([WHITE, 0], [WHITE, 1], 3) \
    .cnot(2, 1) \
    .tof([WHITE, 1], [BLACK, 2], 6).tof([WHITE, 0], [BLACK, 6], 5).tof(0, 6, 7) \
    .cnot(2, 1).cnot(1, 6).cnot(2, 1) \
    .tof([WHITE, 0], [BLACK, 6], 7).tof(0, 6, 3).tof([BLACK, 1], [WHITE, 2], 6) \
    .barrier().circuit
    
draw(unitary_circuit4 , 'unitary4')

### - Example for a = 5

In [237]:
unitary_circuit5 = QCircuit(qubits)

unitary_circuit5 = unitary_circuit5 \
    .tof([WHITE, 1], [BLACK, 2], 7).tof(0, 7, 3) \
    .tof([BLACK, 1], [WHITE, 2], 5).tof(0, 5, 7) \
    .cnot(2, 1) \
    .tof([BLACK, 0], [WHITE, 1], 5) \
    .cnot([WHITE, 1], 3).cnot(2, 1) \
    .barrier().circuit

draw(unitary_circuit5, 'unitary5')

### - Example for a = 8

In [243]:
unitary_circuit8 = QCircuit(qubits)

unitary_circuit8 = unitary_circuit8 \
    .cnot(0, 6).cnot([WHITE, 0], 3) \
    .barrier().circuit

draw(unitary_circuit8, 'unitary8')

### - Example for a = 10

In [248]:
origin_unitary_circuit10 = create_empty_circuit()

origin_unitary_circuit10 << X(qubits[1]) << Toffoli(qubits[1], qubits[2], qubits[3]) << X(qubits[1]) \
                << X(qubits[3]) << Toffoli(qubits[0], qubits[3], qubits[6]) << X(qubits[3]) \
                << Toffoli(qubits[0], qubits[3], qubits[7]) \
                << CNOT(qubits[2], qubits[1]) << CNOT(qubits[6], qubits[4]) << CNOT(qubits[1], qubits[3]) \
                << CNOT(qubits[2], qubits[1]) << X(qubits[0]) \
                << Toffoli(qubits[0], qubits[3], qubits[7]) << X(qubits[0]) \
                << X(qubits[2]) << Toffoli(qubits[1], qubits[2], qubits[3]) << X(qubits[2]) \
                << CNOT(qubits[7], qubits[5]) << CNOT(qubits[2], qubits[1]) << CNOT(qubits[1], qubits[5]) \
                << Toffoli(qubits[0], qubits[1], qubits[4]) << CNOT(qubits[1], qubits[0]) \
                << X(qubits[0]) << CNOT(qubits[0], qubits[3]) << X(qubits[0]) \
                << CNOT(qubits[1], qubits[0]) << CNOT(qubits[2], qubits[1])

origin_unitary_circuit10 << BARRIER(qubits)

# draw(origin_unitary_circuit10, 'unitary10')

In [251]:
unitary_circuit10 = QCircuit(qubits)

unitary_circuit10 = unitary_circuit10 \
    .tof([WHITE, 1], [BLACK, 2], 3).tof([BLACK, 0], [WHITE, 3], 6).tof(0, 3, 7) \
    .cnot(2, 1).cnot(6, 4).cnot(1, 3).cnot(2, 1) \
    .tof([WHITE, 0], [BLACK, 3], 7).tof([BLACK, 1], [WHITE, 2], 3) \
    .cnot(7, 5).cnot(2, 1).cnot(1, 5) \
    .tof(0, 1, 4) \
    .cnot(1, 0).cnot([WHITE, 0], 3).cnot(1, 0).cnot(2, 1) \
    .barrier().circuit

draw(unitary_circuit10, 'unitary10')

### - Example for a = 11

In [257]:
unitary_circuit11 = QCircuit(qubits)

unitary_circuit11 = unitary_circuit11 \
    .tof([BLACK, 1], [WHITE, 2], 3).tof([WHITE, 0], [BLACK, 3], 7).tof([BLACK, 0], [WHITE, 3], 4) \
    .cnot(2, 1).cnot(1, 3).cnot(2, 1) \
    .tof([WHITE, 0], [BLACK, 3], 5).tof([BLACK, 0], [WHITE, 3], 6).tof([WHITE, 1], [BLACK, 2], 3) \
    .cnot(2, 1).cnot([WHITE, 1], 3).cnot(2, 1) \
    .barrier().circuit

draw(unitary_circuit11, 'unitary11')

### - Example for a = 13

In [261]:
unitary_circuit13 = QCircuit(qubits)

unitary_circuit13 = unitary_circuit13 \
    .cnot(0, 6).cnot(0, 5).x(3) \
    .barrier().circuit

draw(unitary_circuit13, 'unitary13')

### - Example for a = 16

In [268]:
unitary_circuit16 = QCircuit(qubits)

unitary_circuit16 = unitary_circuit16 \
    .tof([WHITE, 1], [BLACK, 2], 6) \
    .cnot(2, 1).cnot(6, 4).cnot(1, 4) \
    .tof([WHITE, 0], [BLACK, 6], 7).tof([BLACK, 0], [WHITE, 1], 7) \
    .tof([WHITE, 0], [BLACK, 4], 5).tof(0, 6, 5) \
    .tof([WHITE, 0], [WHITE, 1], 3).tof(0, 4, 3) \
    .cnot(1, 4).cnot(2, 1).cnot(6, 4) \
    .tof([WHITE, 1], [BLACK, 2], 6) \
    .barrier().circuit

draw(unitary_circuit16, 'unitary16')

### - Example for a = 17

In [275]:
unitary_circuit17 = QCircuit(qubits)

unitary_circuit17 = unitary_circuit17 \
    .tof([BLACK, 1], [WHITE, 2], 6) \
    .cnot(2, 1).cnot(6, 4).cnot(1, 4) \
    .tof([BLACK, 0], [WHITE, 1], 7) \
    .cnot([WHITE, 1], 3).cnot(4, 5).cnot(6, 7) \
    .tof(0, 6, 5).tof(0, 4, 3) \
    .cnot(1, 4).cnot(2, 1).cnot(6, 4) \
    .tof([BLACK, 1], [WHITE, 2], 6) \
    .barrier().circuit

draw(unitary_circuit17, 'unitary17')

### - Example for a = 19

In [279]:
unitary_circuit19 = QCircuit(qubits)

unitary_circuit19 = unitary_circuit19 \
    .tof([BLACK, 1], [WHITE, 2], 5).tof(0, 5, 3) \
    .cnot(2, 1).cnot(3, 4).cnot(5, 6) \
    .cnot(0, 4).cnot([WHITE, 1], 3).cnot(1, 6) \
    .tof([WHITE, 0], [BLACK, 6], 7).tof([BLACK, 0], [WHITE, 1], 7) \
    .cnot(1, 6) \
    .tof(0, 1, 6) \
    .cnot(2, 1).cnot(5, 6) \
    .barrier().circuit

draw(unitary_circuit19, 'unitary19')

In [296]:
unitary_circuit19_approximate = QCircuit(qubits)
unitary_circuit19_approximate.expand = 'approximate'

unitary_circuit19_approximate = unitary_circuit19_approximate \
    .tof([BLACK, 1], [WHITE, 2], 5).tof(0, 5, 3) \
    .cnot(2, 1).cnot(3, 4).cnot(5, 6) \
    .cnot(0, 4).cnot([WHITE, 1], 3).cnot(1, 6) \
    .tof([WHITE, 0], [BLACK, 6], 7).tof([BLACK, 0], [WHITE, 1], 7) \
    .cnot(1, 6) \
    .tof(0, 1, 6) \
    .cnot(2, 1).cnot(5, 6) \
    .barrier().circuit

draw(unitary_circuit19_approximate, 'unitary19_approximate')

### - Example for a = 20

In [282]:
unitary_circuit20 = QCircuit(qubits)

unitary_circuit20 = unitary_circuit20 \
    .cnot(0, 7).cnot(0, 5).cnot([WHITE, 0], 3) \
    .barrier().circuit

draw(unitary_circuit20, 'unitary20')

### - Third Step: apply QFT_dagger

In [10]:
qft_dagger_circuit = create_empty_circuit()

for i in range(phase_bits_qubits - 1):
    qft_dagger_circuit << H(qubits[i])
    
    for j in range(i + 1, phase_bits_qubits):
        qft_dagger_circuit << U1(qubits[i], np.pi / (2 ** (j - i))).control(qubits[j])
    
    qft_dagger_circuit << BARRIER(qubits)## Measure and calculate factors

qft_dagger_circuit << H(qubits[phase_bits_qubits - 1])

draw(qft_dagger_circuit, 'qft_dagger')

### - build full circuit program

In [216]:
prog2 = create_empty_qprog()
prog2 << hadamard_circuit << unitary_circuit2 << qft_dagger_circuit

draw(prog2, 'shor_a=2')

In [228]:
prog4 = create_empty_qprog()
prog4 << hadamard_circuit << unitary_circuit4 << qft_dagger_circuit

draw(prog4, 'shor_a=4')

In [238]:
prog5 = create_empty_qprog()
prog5 << hadamard_circuit << unitary_circuit5 << qft_dagger_circuit

draw(prog5, 'shor_a=5')

In [244]:
prog8 = create_empty_qprog()
prog8 << hadamard_circuit << unitary_circuit8 << qft_dagger_circuit

draw(prog8, 'shor_a=8')

In [252]:
prog10 = create_empty_qprog()
prog10 << hadamard_circuit << unitary_circuit10 << qft_dagger_circuit

draw(prog10, 'shor_a=10')

In [258]:
prog11 = create_empty_qprog()
prog11 << hadamard_circuit << unitary_circuit11 << qft_dagger_circuit

draw(prog11, 'shor_a=11')

In [262]:
prog13 = create_empty_qprog()
prog13 << hadamard_circuit << unitary_circuit13 << qft_dagger_circuit

draw(prog13, 'shor_a=13')

In [269]:
prog16 = create_empty_qprog()
prog16 << hadamard_circuit << unitary_circuit16 << qft_dagger_circuit

draw(prog16, 'shor_a=16')

In [276]:
prog17 = create_empty_qprog()
prog17 << hadamard_circuit << unitary_circuit17 << qft_dagger_circuit

draw(prog17, 'shor_a=17')

In [280]:
prog19 = create_empty_qprog()
prog19 << hadamard_circuit << unitary_circuit19 << qft_dagger_circuit

draw(prog19, 'shor_a=19')

In [297]:
prog19_approximate = create_empty_qprog()
prog19_approximate << hadamard_circuit << unitary_circuit19_approximate << qft_dagger_circuit

draw(prog19_approximate, 'shor_a=19_approximate')

In [283]:
prog20 = create_empty_qprog()
prog20 << hadamard_circuit << unitary_circuit20 << qft_dagger_circuit

draw(prog20, 'shor_a=20')

## Measure and calculate factors

In [15]:
def measure(prog):
    for i in range(phase_bits_qubits):
        prog << Measure(qubits[i], cbits[phase_bits_qubits - i - 1])

    result = run_with_configuration(prog, cbits, 1)
    print(f'  result: {result}')
    
    return result

In [16]:
def QPE(result):
    phase = int(list(result)[0], 2) / (2 ** phase_bits_qubits)
    print(f'   - corresponding phase: {phase}')
    
    return phase

In [72]:
def calculatePeriod(phase, a):
    limit = N

    while True:
        frac = Fraction(phase).limit_denominator(limit)
        r = frac.denominator

        # check period
        if (a ** r) % N == 1:
            break
        else:
            limit = r - 1
            if limit <= 0:
                print(f'\n  Rewrite to fraction: {frac}, find period failed')
                return False, None

    print(f'\n  Rewrite to fraction: {frac}, thus r = {r}')

    return True, r

In [79]:
def calculateFactors(r, a):
    if int(math.sqrt(a)) ** 2 == a:
        guesses = [math.gcd(int(a ** (r / 2)) - 1, N), math.gcd(int(a ** (r / 2)) + 1, N)]
    else:
        guesses = [math.gcd(a ** (r // 2) - 1, N), math.gcd(a ** (r // 2) + 1, N)]
    
    print(f'  calculate final guesses: {guesses}')
    
    factors = []
    for num in guesses:
        if num not in [1, N]:
            factors.append(num)
            print(f'[Find non-trivial factor: {num}]')
    
    if len(factors) == 0:
        print('[Failed]')
        return False, []
    elif len(factors) == 1:
        factors.append(N // factors[0])
    
    return True, sorted(factors)

### - final execute

In [312]:
def execute(prog, a):
    max_attempt = 30
    attempt = 1
    
    while True:
        print(f'Execute shor algorithm - attempt time: [{attempt}]')

        result = measure(prog)
        phase = QPE(result)

        # period
        ok, r = calculatePeriod(phase, a)
        if ok:
            
            ok, factors = calculateFactors(r, a)
            if ok:
                print(f'\nFactors of {N} are: {factors} (total run times: {attempt})')
                break
            
        attempt += 1
        if attempt > max_attempt:
            print('\n[FAIL]')
            break
        print('\n' + '-' * 36 + '\n')

In [225]:
execute(prog2, a=2)

Execute shor algorithm - attempt time: [1]
  result: {'001': 1}
   - corresponding phase: 0.125

  Rewrite to fraction: 1/6, thus r = 6
  calculate final guesses: [7, 3]
[Find non-trivial factor: 7]
[Find non-trivial factor: 3]

Factors of 21 are: [3, 7] (total run times: 1)


In [234]:
execute(prog4, a=4)

Execute shor algorithm - attempt time: [1]
  result: {'101': 1}
   - corresponding phase: 0.625

  Rewrite to fraction: 2/3, thus r = 3
  calculate final guesses: [7, 3]
[Find non-trivial factor: 7]
[Find non-trivial factor: 3]

Factors of 21 are: [3, 7] (total run times: 1)


In [241]:
execute(prog5, a=5)

Execute shor algorithm - attempt time: [1]
  result: {'000': 1}
   - corresponding phase: 0.0

  Rewrite to fraction: 0, find period failed

------------------------------------

Execute shor algorithm - attempt time: [2]
  result: {'011': 1}
   - corresponding phase: 0.375

  Rewrite to fraction: 0, find period failed

------------------------------------

Execute shor algorithm - attempt time: [3]
  result: {'011': 1}
   - corresponding phase: 0.375

  Rewrite to fraction: 0, find period failed

------------------------------------

Execute shor algorithm - attempt time: [4]
  result: {'001': 1}
   - corresponding phase: 0.125

  Rewrite to fraction: 1/6, thus r = 6
  calculate final guesses: [1, 21]
[Failed]

------------------------------------

Execute shor algorithm - attempt time: [5]
  result: {'101': 1}
   - corresponding phase: 0.625

  Rewrite to fraction: 1, find period failed

------------------------------------

Execute shor algorithm - attempt time: [6]
  result: {'101'

In [245]:
execute(prog8, a=8)

Execute shor algorithm - attempt time: [1]
  result: {'100': 1}
   - corresponding phase: 0.5

  Rewrite to fraction: 1/2, thus r = 2
  calculate final guesses: [7, 3]
[Find non-trivial factor: 7]
[Find non-trivial factor: 3]

Factors of 21 are: [3, 7] (total run times: 1)


In [253]:
execute(prog10, a=10)

Execute shor algorithm - attempt time: [1]
  result: {'010': 1}
   - corresponding phase: 0.25

  Rewrite to fraction: 0, find period failed

------------------------------------

Execute shor algorithm - attempt time: [2]
  result: {'101': 1}
   - corresponding phase: 0.625

  Rewrite to fraction: 1, find period failed

------------------------------------

Execute shor algorithm - attempt time: [3]
  result: {'001': 1}
   - corresponding phase: 0.125

  Rewrite to fraction: 1/6, thus r = 6
  calculate final guesses: [3, 7]
[Find non-trivial factor: 3]
[Find non-trivial factor: 7]

Factors of 21 are: [3, 7] (total run times: 3)


In [259]:
execute(prog11, a=11)

Execute shor algorithm - attempt time: [1]
  result: {'101': 1}
   - corresponding phase: 0.625

  Rewrite to fraction: 1, find period failed

------------------------------------

Execute shor algorithm - attempt time: [2]
  result: {'010': 1}
   - corresponding phase: 0.25

  Rewrite to fraction: 0, find period failed

------------------------------------

Execute shor algorithm - attempt time: [3]
  result: {'001': 1}
   - corresponding phase: 0.125

  Rewrite to fraction: 1/6, thus r = 6
  calculate final guesses: [7, 3]
[Find non-trivial factor: 7]
[Find non-trivial factor: 3]

Factors of 21 are: [3, 7] (total run times: 3)


In [267]:
execute(prog13, a=13)

Execute shor algorithm - attempt time: [1]
  result: {'100': 1}
   - corresponding phase: 0.5

  Rewrite to fraction: 1/2, thus r = 2
  calculate final guesses: [3, 7]
[Find non-trivial factor: 3]
[Find non-trivial factor: 7]

Factors of 21 are: [3, 7] (total run times: 1)


In [270]:
execute(prog16, a=16)

Execute shor algorithm - attempt time: [1]
  result: {'000': 1}
   - corresponding phase: 0.0

  Rewrite to fraction: 0, find period failed

------------------------------------

Execute shor algorithm - attempt time: [2]
  result: {'110': 1}
   - corresponding phase: 0.75

  Rewrite to fraction: 2/3, thus r = 3
  calculate final guesses: [21, 1]
[Failed]

------------------------------------

Execute shor algorithm - attempt time: [3]
  result: {'101': 1}
   - corresponding phase: 0.625

  Rewrite to fraction: 2/3, thus r = 3
  calculate final guesses: [21, 1]
[Failed]

------------------------------------

Execute shor algorithm - attempt time: [4]
  result: {'110': 1}
   - corresponding phase: 0.75

  Rewrite to fraction: 2/3, thus r = 3
  calculate final guesses: [21, 1]
[Failed]

------------------------------------

Execute shor algorithm - attempt time: [5]
  result: {'110': 1}
   - corresponding phase: 0.75

  Rewrite to fraction: 2/3, thus r = 3
  calculate final guesses: [21,

In [277]:
execute(prog17, a=17)

Execute shor algorithm - attempt time: [1]
  result: {'101': 1}
   - corresponding phase: 0.625

  Rewrite to fraction: 1, find period failed

------------------------------------

Execute shor algorithm - attempt time: [2]
  result: {'000': 1}
   - corresponding phase: 0.0

  Rewrite to fraction: 0, find period failed

------------------------------------

Execute shor algorithm - attempt time: [3]
  result: {'110': 1}
   - corresponding phase: 0.75

  Rewrite to fraction: 1, find period failed

------------------------------------

Execute shor algorithm - attempt time: [4]
  result: {'011': 1}
   - corresponding phase: 0.375

  Rewrite to fraction: 0, find period failed

------------------------------------

Execute shor algorithm - attempt time: [5]
  result: {'010': 1}
   - corresponding phase: 0.25

  Rewrite to fraction: 0, find period failed

------------------------------------

Execute shor algorithm - attempt time: [6]
  result: {'111': 1}
   - corresponding phase: 0.875

  

In [281]:
execute(prog19, a=19)

Execute shor algorithm - attempt time: [1]
  result: {'101': 1}
   - corresponding phase: 0.625

  Rewrite to fraction: 1, find period failed

------------------------------------

Execute shor algorithm - attempt time: [2]
  result: {'001': 1}
   - corresponding phase: 0.125

  Rewrite to fraction: 1/6, thus r = 6
  calculate final guesses: [3, 7]
[Find non-trivial factor: 3]
[Find non-trivial factor: 7]

Factors of 21 are: [3, 7] (total run times: 2)


In [328]:
execute(prog19_approximate, a=19)

Execute shor algorithm - attempt time: [1]
  result: {'110': 1}
   - corresponding phase: 0.75

  Rewrite to fraction: 1, find period failed

------------------------------------

Execute shor algorithm - attempt time: [2]
  result: {'010': 1}
   - corresponding phase: 0.25

  Rewrite to fraction: 0, find period failed

------------------------------------

Execute shor algorithm - attempt time: [3]
  result: {'110': 1}
   - corresponding phase: 0.75

  Rewrite to fraction: 1, find period failed

------------------------------------

Execute shor algorithm - attempt time: [4]
  result: {'000': 1}
   - corresponding phase: 0.0

  Rewrite to fraction: 0, find period failed

------------------------------------

Execute shor algorithm - attempt time: [5]
  result: {'101': 1}
   - corresponding phase: 0.625

  Rewrite to fraction: 1, find period failed

------------------------------------

Execute shor algorithm - attempt time: [6]
  result: {'101': 1}
   - corresponding phase: 0.625

  R

In [324]:
execute(prog20, a=20)

Execute shor algorithm - attempt time: [1]
  result: {'000': 1}
   - corresponding phase: 0.0

  Rewrite to fraction: 0, find period failed

------------------------------------

Execute shor algorithm - attempt time: [2]
  result: {'000': 1}
   - corresponding phase: 0.0

  Rewrite to fraction: 0, find period failed

------------------------------------

Execute shor algorithm - attempt time: [3]
  result: {'111': 1}
   - corresponding phase: 0.875

  Rewrite to fraction: 7/8, thus r = 8
  calculate final guesses: [21, 1]
[Failed]

------------------------------------

Execute shor algorithm - attempt time: [4]
  result: {'111': 1}
   - corresponding phase: 0.875

  Rewrite to fraction: 7/8, thus r = 8
  calculate final guesses: [21, 1]
[Failed]

------------------------------------

Execute shor algorithm - attempt time: [5]
  result: {'100': 1}
   - corresponding phase: 0.5

  Rewrite to fraction: 1/2, thus r = 2
  calculate final guesses: [1, 21]
[Failed]

-------------------------